# Clone WaveRNN repo

In [1]:
# Clone the repo
!git clone https://github.com/as-ideas/ForwardTacotron
# Checkout to latest version as of the time this notebook was created
!cd ForwardTacotron && git reset --hard "f537c5f16f9459948b64f563749d65dcedacbe04"

Cloning into 'ForwardTacotron'...
remote: Enumerating objects: 5473, done.
remote: Counting objects: 100% (1844/1844), done.
remote: Compressing objects: 100% (744/744), done.
remote: Total 5473 (delta 1342), reused 1512 (delta 1095), pack-reused 3629
Receiving objects: 100% (5473/5473), 205.55 MiB | 22.36 MiB/s, done.
Resolving deltas: 100% (3752/3752), done.
HEAD is now at f537c5f Update README.md


# Download data

In [2]:
Size = "Smaller" #@param ["Full", "Small", "Smaller"]
if Size == "Full":
  vctk_dataset_path = "/content/VCTK-Corpus/"
  archive_path="/content/VCTK-Corpus.tar.gz"
  !test ! -e $archive_path && wget -O $archive_path "https://queensuca-my.sharepoint.com/:u:/g/personal/16drp4_queensu_ca/EcnCAgHdAwBJmWk9vNcpiKgB5Rhi6GNtSchx6zGEhWe-fw?download=1"
  !test ! -e $vctk_dataset_path && tar -xf $archive_path -O $vctk_dataset_path
elif Size == "Small":
  vctk_dataset_path = "/content/VCTK-Corpus-small/"
  !test ! -e /content/VCTK-Corpus-small.7z && wget -O /content/VCTK-Corpus-small.7z "https://queensuca-my.sharepoint.com/:u:/g/personal/16drp4_queensu_ca/Ebz_WoXK4T9KjbJq_SFCsZQB94PqsKOnQ9HwB5olpGeQIw?download=1"
  !test ! -e $vctk_dataset_path && 7z x VCTK-Corpus-small.7z
elif Size == "Smaller":
  vctk_dataset_path = "/content/VCTK-Corpus-smaller/"
  archive_path="/content/VCTK-Corpus-smaller.tar.gz"
  !test ! -e $archive_path && wget -O $archive_path "https://queensuca-my.sharepoint.com/:u:/g/personal/16drp4_queensu_ca/EfRSVbO-ohRFlUFhA9lx4qsBVl4s5FU676hF9uTRSAjOgg?download=1"
  !test ! -e $vctk_dataset_path && tar -xzf $archive_path


--2021-12-02 21:14:44--  https://queensuca-my.sharepoint.com/:u:/g/personal/16drp4_queensu_ca/EfRSVbO-ohRFlUFhA9lx4qsBVl4s5FU676hF9uTRSAjOgg?download=1
Resolving queensuca-my.sharepoint.com (queensuca-my.sharepoint.com)... 13.107.136.9, 13.107.138.9
Connecting to queensuca-my.sharepoint.com (queensuca-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/16drp4_queensu_ca/Documents/School%20Files/2021-2023,%20Masters/CISC%20867%20-%20Deep%20Learning/Datasets/VCTK-Corpus-smaller.tar.gz [following]
--2021-12-02 21:14:45--  https://queensuca-my.sharepoint.com/personal/16drp4_queensu_ca/Documents/School%20Files/2021-2023,%20Masters/CISC%20867%20-%20Deep%20Learning/Datasets/VCTK-Corpus-smaller.tar.gz
Reusing existing connection to queensuca-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 556673395 (531M) [application/x-gzip]
Saving to: ‘/content/VCTK-Corpus-smaller.tar.gz’

/content/VCTK-Corpu 1

# Create dataset

In [8]:
import importlib
%cd /content
from Project.data.VCTK import VCTKDataset
# importlib.reload(Project.data.VCTK) # prevent cache when making changes

dataset = VCTKDataset(vctk_dataset_path)
print(len(dataset))
print(dataset[0])

/content


  0%|          | 0/2118 [00:00<?, ?it/s]

2118
('/content/VCTK-Corpus-smaller/txt/p225/p225_001.txt', (array([0.00399593, 0.00702291, 0.00598958, ..., 0.00792369, 0.00879966,
       0.        ], dtype=float32), 22050))
